In [1]:
import geemap,ee
from ployinterp import ployinterp_column

ee.Initialize()

jx = geemap.shp_to_ee(r'C:\Users\xiong\Documents\geemap\shp\boundary\boundary2.shp')

image1 = ee.ImageCollection("MODIS/MCD43A4_006_NDVI")\
                .filterDate('2018-01-01','2018-02-01')\
                .first().clip(jx.geometry())
image2 = ee.ImageCollection("MODIS/MCD43A4_006_NDVI")\
                .filterDate('2019-01-01','2019-02-01')\
                .first().clip(jx.geometry())
image3 = ee.ImageCollection("MODIS/MCD43A4_006_NDVI")\
                .filterDate('2020-01-01','2020-02-01')\
                .first().clip(jx.geometry())
List = [image1,image2]

In [2]:
region = jx.geometry()
num=1
default_value=0

In [3]:
import traceback
import pandas as pd
import numpy as np
from scipy.interpolate import lagrange
import geemap

def ployinterp_column(s, n):
    List = list(range(0, n)) + list(range(n+1, len(s)))
    y = s.values[List] #取数，转换成列表
    List = np.array(List)[~np.isnan(y)]
    y = y[~np.isnan(y)] #剔除空值
    # print("List:{},y:{}".format(List,y))
    #print("表达式:{}".format(lagrange(List, y)))
    return lagrange(List, y)(n) #插值并返回插值结果，n是被插值的位置


arr = geemap.ee_to_numpy(ee_object=List[0],region=region,default_value=default_value)
result = pd.DataFrame(arr.reshape(-1,1))

for image in List[1:]:
    arr_temp = geemap.ee_to_numpy(ee_object=image,region=region,default_value=default_value)
    arr_temp = arr_temp.reshape(-1,1)
    result = pd.concat([result,pd.DataFrame(arr_temp.reshape(-1,1))],axis=1)

result_m = result.mean(axis=1).copy()
result_s = np.std(np.array(result),axis=1)

result_max = pd.DataFrame(result_m)+pd.DataFrame(result_s)*2
result_min = pd.DataFrame(result_m)-pd.DataFrame(result_s)*2
del result_m,result_s

result.columns = [i for i in range(len(List))]

r_handle = result[[num]].copy()

for n, Max, Min, data in zip(result.index, result_max.values, result_min.values, r_handle.values):
    if (data[0] < Min[0]) or (data[0] > Max[0]) or (data[0] == default_value):
        r_handle.iloc[n,0] = default_value
        r_handle.iloc[n,0] = ployinterp_column(result.iloc[n],num)

r_arr = np.array(r_handle).astype(type(List[0])).reshape(arr.shape[0],arr.shape[1])


# numpy to ee
# data = geemap.numpy_to_ee(np_array=r_arr,
#                   crs=List[0].projection().getInfo()['crs'],
#                   transform=List[0].projection().getInfo()['transform'],
#                   band_names=List[0].bandNames().getInfo())

In [7]:
crs=List[0].projection().getInfo()['crs']
transform=List[0].projection().getInfo()['transform']

projection = ee.Projection(crs, transform)
coords = ee.Image.pixelCoordinates(projection).floor().int32()
x = coords.select("x")
y = coords.select("y")
s = r_arr.shape
if len(s) < 3:
    dimx = s[0]
    dimy = s[1]
else:
    dimx = s[1]
    dimy = s[2]
    dimz = s[0]

coord_mask = x.gte(0).And(y.gte(0)).And(x.lt(dimx)).And(y.lt(dimy))
coords = coords.updateMask(coord_mask)